In [1]:
# check version
import sys
print(sys.version)

3.8.12 (default, Oct 12 2021, 13:49:34) 
[GCC 7.5.0]


In [2]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
import numpy as np
import os
import subprocess
import sys
from collections import defaultdict
import PIL
import torch
import torchvision
from tabulate import tabulate

__all__ = ["collect_env_info"]


def collect_torch_env():
    try:
        import torch.__config__

        return torch.__config__.show()
    except ImportError:
        # compatible with older versions of pytorch
        from torch.utils.collect_env import get_pretty_env_info

        return get_pretty_env_info()


def get_env_module():
    var_name = "DETECTRON2_ENV_MODULE"
    return var_name, os.environ.get(var_name, "<not set>")


def collect_env_info():
    data = []
    data.append(("sys.platform", sys.platform))
    data.append(("Python", sys.version.replace("\n", "")))
    data.append(("Numpy", np.__version__))
    try:
        from detectron2 import _C
    except ImportError:
        data.append(("detectron2._C", "failed to import"))
    else:
        data.append(("Detectron2 Compiler", _C.get_compiler_version()))
        data.append(("Detectron2 CUDA Compiler", _C.get_cuda_version()))

    data.append(get_env_module())
    data.append(("PyTorch", torch.__version__))
    data.append(("PyTorch Debug Build", torch.version.debug))
    try:
        data.append(("torchvision", torchvision.__version__))
    except AttributeError:
        data.append(("torchvision", "unknown"))

    has_cuda = torch.cuda.is_available()
    data.append(("CUDA available", has_cuda))
    if has_cuda:
        devices = defaultdict(list)
        for k in range(torch.cuda.device_count()):
            devices[torch.cuda.get_device_name(k)].append(str(k))
        for name, devids in devices.items():
            data.append(("GPU " + ",".join(devids), name))

        from torch.utils.cpp_extension import CUDA_HOME

        data.append(("CUDA_HOME", str(CUDA_HOME)))

        if CUDA_HOME is not None and os.path.isdir(CUDA_HOME):
            try:
                nvcc = os.path.join(CUDA_HOME, "bin", "nvcc")
                nvcc = subprocess.check_output("'{}' -V | tail -n1".format(nvcc), shell=True)
                nvcc = nvcc.decode("utf-8").strip()
            except subprocess.SubprocessError:
                nvcc = "Not Available"
            data.append(("NVCC", nvcc))

        cuda_arch_list = os.environ.get("TORCH_CUDA_ARCH_LIST", None)
        if cuda_arch_list:
            data.append(("TORCH_CUDA_ARCH_LIST", cuda_arch_list))
    data.append(("Pillow", PIL.__version__))

    try:
        import cv2

        data.append(("cv2", cv2.__version__))
    except ImportError:
        pass
    env_str = tabulate(data) + "\n"
    env_str += collect_torch_env()
    return env_str


if __name__ == "__main__":
    print(collect_env_info())


---------------------  ---------------------------------------------------
sys.platform           linux
Python                 3.8.12 (default, Oct 12 2021, 13:49:34) [GCC 7.5.0]
Numpy                  1.23.3
detectron2._C          failed to import
DETECTRON2_ENV_MODULE  <not set>
PyTorch                1.9.0
PyTorch Debug Build    False
torchvision            0.10.0
CUDA available         True
GPU 0                  NVIDIA GeForce RTX 2060
CUDA_HOME              /usr
NVCC                   Cuda compilation tools, release 10.1, V10.1.243
Pillow                 9.2.0
cv2                    4.6.0
---------------------  ---------------------------------------------------
PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU

In [3]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
NVIDIA GeForce RTX 2060, 510.47.03, 6144 MiB


In [4]:
!pip install tensorboardx

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [5]:
%cd /home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement
%ls

/home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement
 config/    experiments/          learn/    nohup.out        'view?usp=sharing'
 core/      I830000_E32_gen.pth   LICENSE   README.md
 data/      I830000_E32_opt.pth   misc/     requirement.txt
 dataset/   infer.py*             model/    sample.py*
 eval.py*   input/                models/   sr.py*


In [7]:
# download weight
# using proxy
!gdown --id 1M015uQG8cq0mqwhEfc7klFJwDr2jsYSr --proxy http://127.0.0.1:7890/


/home/tzr/.local/lib/python3.8/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Using proxy: http://127.0.0.1:7890/
Downloading...
From: https://drive.google.com/uc?id=1M015uQG8cq0mqwhEfc7klFJwDr2jsYSr
To: /home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement/I830000_E32_opt.pth
100%|██████████████████████████████████████| 1.24G/1.24G [02:29<00:00, 8.29MB/s]


In [10]:
!gdown --id 1Vwe1Raa-Dpop75o5v7O5qvGoOdFDlyc- --proxy http://127.0.0.1:7890/


/home/tzr/.local/lib/python3.8/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Using proxy: http://127.0.0.1:7890/
Downloading...
From: https://drive.google.com/uc?id=1Vwe1Raa-Dpop75o5v7O5qvGoOdFDlyc-
To: /home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement/I830000_E32_gen.pth
100%|████████████████████████████████████████| 622M/622M [01:16<00:00, 8.13MB/s]


In [7]:
# create files
%cd /home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement
!python data/prepare_data.py --path /home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement/input --size 64,512 --out ./dataset/celebahq

/home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement
data/prepare_data.py:30: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  def resize_multiple(img, sizes=(16, 128), resample=Image.BICUBIC, lmdb_save=False):
data/prepare_data.py:100: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  def prepare(img_path, out_path, n_worker, sizes=(16, 128), resample=Image.BICUBIC, lmdb_save=False):
data/prepare_data.py:176: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  resample_map = {'bilinear': Image.BILINEAR, 'bicubic': Image.BICUBIC}
data/prepare_data.py:176: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample_map = {'bilinear': Image.BILINEAR, 'bicubic': Image.BICUBIC}

In [9]:
# !ls
# main implement
%cd /home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement
!python infer.py -c /home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement/config/infer.json

/home/tzr/Documents/GitHubSYNC/Image-Super-Resolution-via-Iterative-Refinement
export CUDA_VISIBLE_DEVICES=0,1
22-10-14 19:23:47.001 - INFO:   name: distributed_high_sr_ffhq
  phase: val
  gpu_ids: [0, 1]
  path:[
    log: experiments/distributed_high_sr_ffhq_221014_192347/logs
    tb_logger: experiments/distributed_high_sr_ffhq_221014_192347/tb_logger
    results: experiments/distributed_high_sr_ffhq_221014_192347/results
    checkpoint: experiments/distributed_high_sr_ffhq_221014_192347/checkpoint
    resume_state: I830000_E32
    experiments_root: experiments/distributed_high_sr_ffhq_221014_192347
  ]
  datasets:[
    train:[
      name: FFHQ
      mode: HR
      dataroot: dataset/ffhq_64_512
      datatype: img
      l_resolution: 64
      r_resolution: 512
      batch_size: 2
      num_workers: 8
      use_shuffle: True
      data_len: -1
    ]
    val:[
      name: CelebaHQ
      mode: LRHR
      dataroot: dataset/celebahq_64_512
      datatype: img
      l_resolution: 64
      r